**Put the token in the clipboard before you start**

In [ ]:
cd  "C:\Users\P10506111\Documents\github\vsteam"
ipmo .\dist\VSTeam.psd1 -Force

In [ ]:
Connect-VSTeam -TokenFromClipboard -Account bbcas  #or  EMSAlcoholMonitoring or whatever 

In [ ]:
(Get-VSTeamInfo).account

https://dev.azure.com/bbcas


In [ ]:
Get-VSTeamProject | out-string -width 132


Name                                   Description
----                                   -----------
BBCAS - Interfaces                     
BBCAS - Infrastructure and Environment 
BBCAS - DevOps                         
BBCAS - UAT                            Project for recording issues raised in UAT across the BBCAS Programme
BBC Reports                            
BBCAS - CRM                            BBC Audience Services CRM Development
BBCAS - Data Migration                 
BBCAS Internal Change and Observations BBCAS Internal Change Board 
BBCAS Change and Observations          BBC Change Board
BBCAS - Reporting and BI               Place Holder for Master Copies of all the reports that are built




In [ ]:
#Any from the list above
Set-VSTeamDefaultProject "BBCAS Internal Change and Observations"

In [ ]:
# Picklists change the we're redirecting to as needed
$picklistHash = @{}
Get-VSTeamPickList | Get-VSTeamPickList | ForEach-Object -begin {$i=0} -process {
      $i++; 
      $picklistHash[$_.id] = $i
      '${0,-20}= Add-VsTeamPicklist          -Name {1,-60} -Type {2,-10} -Items @({3})  #ID: {4} ' -f "list$i", "'$($_.name)'", "'$($_.type)'", "'$($_.Items -join ''', ''')'", $($_.id)
} > ~\export2.ps1

In [ ]:
#fields. Again: change the destination file
$fieldHash = @{}
Get-VSTeamField | Where-Object referenceName -NotMatch "^System|^Microsoft" | ForEach-Object -begin {$i=0} -process {
      $i ++;
      if ($_.isIdentity) {$type = "'identity'"} else {$type = "'$($_.type)'"}
      $cmdline = '${0,-20}= Add-VSTeamField             -Name {1,-60} -Type {2,-10} -Usage {3,-33} -ReferenceName {4,-74}  ' -f "field$i"  , "'$($_.name)'", $type,"'$($_.usage)'", "'$($_.referenceName)'"
      $fieldHash[$_.referenceName] = "`$field$i"
      if ($_.description)              {$cmdline += " -Description '$($_.description)' "}
      if ($_.readOnly)                 {$cmdline += " -ReadOnly "}
      if (-not $_.isQueryable)         {$cmdline += " -NotQueryable "}
      if (-not $_.canSortBy)           {$cmdline += " -NoSort "}
      if ($_.picklistId)    {           $cmdline += " -PicklistID  `$list$($picklistHash[$_.picklistId]).id " 
         if ($_.isPicklistSuggested)   {$cmdline += " -IsPickListSuggested "}
      $cmdline += "   # Old picklistID was $($_.picklistId)"
      }
      $cmdline   >> ~\export2.ps1
}

In [ ]:
#Processes and work item types. Again check we're going to the right file. 
$processes = Get-VSTeamProcess 
$processhash = @{} ; foreach ($p in $processes) {$processhash[$p.id]=$p.name}
$o = $n =  $m = $l = $k= $j = $i = $h = 0
$result =  foreach ($p in $processes.where({$_.type -eq 'inherited'})) {
   "`r`n########## Process template: '$($p.name) "
   $i ++
   $cmd = '${0,-20}= Add-VSTeamProcess           -ParentProcessName {2,-47} -ProcessTemplate {1,-40}'  -f "process$i", "'$($p.Name)'","'$($processhash[$p.ParentProcessTypeId])'"
   if ($p.referenceName -notmatch '^Inherited\.')  {
         $cmd += ' -ReferenceName {0,-40} ' -f "'$($p.referenceName)'" 
   }
   if ($p.Description)  {
         $cmd += " -Description '$($p.Description)'" 
   }
   $cmd
   #Get the work item types bound to the process. 
   $wiTypes = Get-VSTeamWorkItemType     -ProcessTemplate $p.name | where-object referenceName -notmatch "^microsoft" 
   foreach ($w in $wiTypes) {
      Write-Progress "Dumping Devops processinformation" -CurrentOperation "Dumping Process $($p.name), work item $($w.name)"
      $j ++ ;
      '#region {1} {0}' -f  $w.name, $p.name
      $cmd = '${0,-20}= Add-VSTeamWorkItemType      -Name {1,-60} -ProcessTemplate {2,-40} -Icon          {3,-40} -Color         {4,-20}'  -f "ItemType$j",  "'$($w.name)'", "`$Process$i.name", "'$($w.icon)'", "'$($w.color)'"
      if ($W.description) {$cmd + " -Description   '$($w.description)'"}
      else                {$cmd}

      Get-VsteamWorkItemState     -ProcessTemplate $p.name -WorkItemType $w.name |
         Where-Object {$_customizationType -ne 'system' -and $_.name -notin @('Proposed','Active','Closed') } | ForEach-Object {
         $k++
         '${0,-20}= Add-VsteamWorkItemState     -Name {1,-60} -ProcessTemplate {2,-40} -WorkItemType  {3,-40} -Color         {4,-20} -StateCategory {5,-25} -Order {6}'   -f "State$k", "'$($_.Name)'", "`$Process$i.name", "`$ItemType$j.name", "'$($_.color)'", "'$($_.StateCategory)'",$_.order
      }
      #Fields in the organization which are part of this work item
      $workItemfieldHash = @{}
      Get-VsteamWorkItemField     -ProcessTemplate $p.name -WorkItemType $w.name | where-object customization -notmatch 'system|inherited' | ForEach-Object {
         $l ++;
         if ($fieldHash[$_.referenceName]) {$field = $fieldHash[$_.referenceName] + '.referenceName'} else {$field = "'$($_.referenceName)'" }
         $cmd = '${0,-20}= Add-VsteamWorkItemField     -Name {1,-60} -ProcessTemplate {2,-40} -WorkItemType  {3,-40} -DefaultValue  {4,-20}'  -f "workitemField$l", $field, "`$Process$i.name", "`$ItemType$j.name", "'$($_.defaultValue)'"
         if ($_.required)    {$cmd += ' -Required ' }
         if ($_.ReadOnly)    {$cmd += ' -ReadOnly ' }
         if ($_.AllowGroups) {$cmd += ' -AllowGroups '}
         "$cmd  # $($_.referenceName)"
         $workItemfieldHash[$_.referenceName] = "`$workitemField$l"
      }

      $pages = Get-VsteamWorkItemPage  -ProcessTemplate $p.name -WorkItemType $w.name
      Write-Progress "Dumping Devops processinformation" -CurrentOperation "Dumping Process $($p.name), work item $($w.name) layout"
      foreach ($pg in $pages) {
         if ($pg.inherited)  {$page = "'$($pg.label)'"}
         else {
            $m ++ 
            '${0,-20}= Add-VsteamWorkItemPage      -Name {1,-60} -ProcessTemplate {2,-40} -WorkItemType  {3,-40} -PageType      {4,-20}'  -f "Page$m", "'$($pg.label)'", "`$Process$i.name", "`$ItemType$j.name", "'$($pg.pageType)'"
            $page = "`$page$m.PageLabel"
         } 
      
         foreach ($sec in $pg.sections) {foreach ($grp in $sec.groups) {
            if ($grp.inherited -or $grp.controls[0].controltype -eq 'HtmlFieldControl' ) { $group = "'$($grp.label)'"} 
            else {
               $n ++  
               '${0,-20}= Add-VSTeamWorkItemPageGroup -Name {1,-60} -ProcessTemplate {2,-40} -WorkItemType  {3,-40} -Page          {4,-20} -Section       {5}'  -f "PageGroup$n", "'$($grp.label)'", "`$Process$i.name", "`$ItemType$j.name", $page , "'$($sec.id)'"
               $group = "`$PageGroup$n.label"
            }  
         
            foreach ($co in $grp.controls) {
               if (-not $co.inherited)  {
                  $o ++ 
                  if ($workItemfieldHash[$co.id]) {$field = $workItemfieldHash[$co.id] + '.referenceName'} else {$field = "'$($co.id)'" }
                  if ($co.controlType -eq 'HtmlFieldControl') {$destProp = '-Section'; $destValue = "'$($sec.id)'"}
                  else                                        {$destProp = '-Group  '; $destValue =  $group       }      
                  $cmd = '${0,-20}= Add-VSTeamWorkItemControl   -Name {1,-60} -ProcessTemplate {2,-40} -WorkItemType  {3,-40} -Page          {4,-20} {7}       {5,-25} -Label {6}'  -f "control$o", $field, "`$Process$i.name",  "`$ItemType$j.name", "'$($pg.label)'", $destValue, "'$($co.label)'", $destprop
                  if (-not $co.visible)    {$cmd + '  -Hidden'} 
                  elseif ($co.overridden)  {$cmd + '  -Hidden:$false'} 
                  else                     {$cmd}
               }
               if ($co.inherited -and -not $co.visible) {
                  $o ++
                  '${0,-20}= Set-VSTeamWorkItemControl   -Hide {1,-60} -ProcessTemplate {2,-40} -WorkItemType  {3,-40} -Page          {4,-20} -Group         {5,-25} -Label {6}'  -f "control$o","", "`$Process$i.name",  "`$ItemType$j.name", "'$($pg.label)'", $group, "'$($co.id)'"
               }
            }
         }}
      }
      $b = $null
      $b = Get-VSTeamWorkItemBehavior -ProcessTemplate $p.Name -WorkItemType $w.name -WarningAction SilentlyContinue
      if ($b.behavior.id -match "requirement") {
         $h ++
         $cmd = '${0,-20}= Set-VSTeamWorkItemBehavior  -WorkItemType {1,-52} -ProcessTemplate {2,-40} -Behavior Requirements -force'  -f "Behavior$h",  "'$($w.name)'", "`$Process$i.name"
         if ($b.isDefault) { $cmd = $cmd + ' -IsDefault' }
         $cmd 
      } 

      if ($w.inherits -like 'Microsoft*' -and $w.isDisabled) {
         '${0,-20}= Set-VSTeamWorkItemType      -Name {1,-60} -ProcessTemplate {2,-40} -Disabled -Force'  -f "ItemType$j",  "`$ItemType$j.name", "`$Process$i.name"
      }
      "#endregion"
   }
}
$result >>  ~\export2.ps1
1..$h | %{"`$Behavior$_"} >>  ~\export2.ps1
1..$k | %{"`$State$_"}     >>  ~\export2.ps1
1..$o | %{"`$control$_"}  >>  ~\export2.ps1


In [ ]:
   Get-VsteamWorkItemState  -ProcessTemplate 'CAS CMMI'-WorkItemType "observations" | Where-Object {$_customizationType -ne 'system' -and $_.name -notin @('Proposed','Active','Closed') }

WorkItemType Order Name     Category Color  Customization Hidden


------------ ----- ----     -------- -----  ------------- ------


Observations 2     On Hold  Proposed D5D5D5 custom        


Observations 4     Resolved Resolved ff9d00 custom        


Observations 5     Removed  Resolved FF9D00 custom        


In [ ]:
$grp = $pages[0].sections[0].groups[1]
$co = $grp.controls[0] 
$co.controltype -ne 'HtmlFieldControl'

False


In [ ]:
(Get-VSTeamWorkItemType -ProcessTemplate 'CAS CMMI' -WorkItemType "impact assessment" | Get-VSTeamWorkItemPage).sections.groups[-3].controls.where({$_.inherited -and -not $_.visible}) | fl * 

id              : Related Work


inherited       : True


overridden      : True


label           : 


controlType     : LinksControl


...

GroupLabel      : Related Work


SectionID       : Section3


PageLabel       : Impact Assesment


ProcessTemplate : CAS CMMI


WorkItemType    : Impact Assessment


In [ ]:
$b.behavior.id -match "requirement"

True


In [ ]:
$b.isDefault

False


In [ ]:
 $processbehaviors = Get-VSTeamProcessBehavior -ProcessTemplate $p 

In [ ]:
$processbehaviors.name

Epics


Features


Backlog items


Tasks


Portfolio


Ordered
